# Homework 4 - Computing Point-in-Time Residual Returns
In this homework, we will use regressions to compute beta-adjusted "residual" returns in a point-in-time fashion suitable for backtesting / live trading.


1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.
2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.
3. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".
4. Compare the volatility of the residual returns to that of the original returns. What do you notice?
5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?
6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio.


In [5]:
import yfinance as yf
import statsmodels.api as sm


In [6]:
tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'QQQ']
data = yf.download(tickers, start='2016-01-01')

[*********************100%%**********************]  6 of 6 completed


In [7]:
adj_close = data['Adj Close']

In [8]:
ret = adj_close/adj_close.shift() -1 

In [96]:
Y = ret[['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']]
Y = Y.dropna()
X = ret[['QQQ']]
X = sm.add_constant(X)
X = X.dropna()
def ols(Y, X):
    return sm.OLS(Y, X).fit().params.loc['QQQ']
beta = Y.rolling(252).apply(
    lambda y: ols(y, X.loc[y.index])
)


In [98]:
beta_contrib = beta.mul(X['QQQ'], axis = 0)
print(beta_contrib)

Ticker          META      AAPL      AMZN      NFLX     GOOGL
Date                                                        
2016-01-05       NaN       NaN       NaN       NaN       NaN
2016-01-06       NaN       NaN       NaN       NaN       NaN
2016-01-07       NaN       NaN       NaN       NaN       NaN
2016-01-08       NaN       NaN       NaN       NaN       NaN
2016-01-11       NaN       NaN       NaN       NaN       NaN
...              ...       ...       ...       ...       ...
2024-07-31  0.042115  0.024528  0.033609  0.027360  0.030583
2024-08-01 -0.032707 -0.019994 -0.027221 -0.022042 -0.024568
2024-08-02 -0.031853 -0.019086 -0.027928 -0.021517 -0.024038
2024-08-05 -0.039575 -0.024731 -0.035297 -0.026989 -0.030667
2024-08-06  0.012751  0.007794  0.011454  0.008684  0.009784

[2161 rows x 5 columns]


In [100]:
resid_return = Y - beta_contrib
print(resid_return)

Ticker          META      AAPL      AMZN      NFLX     GOOGL
Date                                                        
2016-01-05       NaN       NaN       NaN       NaN       NaN
2016-01-06       NaN       NaN       NaN       NaN       NaN
2016-01-07       NaN       NaN       NaN       NaN       NaN
2016-01-08       NaN       NaN       NaN       NaN       NaN
2016-01-11       NaN       NaN       NaN       NaN       NaN
...              ...       ...       ...       ...       ...
2024-07-31 -0.016985 -0.009538 -0.004607 -0.018092 -0.023243
2024-08-01  0.080956  0.003243  0.011658  0.016472  0.020021
2024-08-02  0.012566  0.025955 -0.059919  0.003577  0.000028
2024-08-05  0.014152 -0.023436 -0.005680  0.002398 -0.013795
2024-08-06  0.025843 -0.017542 -0.005802  0.009727 -0.015812

[2161 rows x 5 columns]


In [110]:
filtered_2024 = resid_return['AAPL'].loc[resid_return.index.year == 2024]